# Training the Neural Networks

The pretrained neural networks are available as part of the download from notebook [2. Dataset Generation](2.%20Dataset%20Generation.ipynb). If you haven't downloaded the models from the release yet, we suggest that you do now.

In case you want to train your own models, here is the code to do so.

In [1]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH "] = "true"

import arrow
import tensorflow as tf
from tqdm.notebook import tqdm

from deepalign import Dataset
from deepalign import fs
from deepalign.alignments import ConfNet

We can get all dataset filenames using this helper method.

In [2]:
datasets = sorted([f.name for f in fs.get_event_log_files()])

Now we can train a ConfNet (name of the RNN model) model for each of the datasets using the following for loop. It will create a version of ConfNet with no attributes `(0, 0)`, only case attributes `(0, 1)`, only event attributes `(1, 0)`, and both `(1, 1)`.

In [ ]:
for dataset_name in datasets:
    for ea, ca in [(0, 0), (0, 1), (1, 0), (1, 1) ]: # (0, 0), (0, 1), (1, 0), (1, 1) 
        start_time = arrow.now()
        dataset = Dataset(dataset_name, use_case_attributes=ca, use_event_attributes=ea)
        if ca and dataset.num_case_attributes == 0:
            continue
        confnet = ConfNet(dataset, use_case_attributes=ca, use_event_attributes=ea)
        confnet.fit(dataset, batch_size=100, epochs=50, validation_split=0.1,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)])
        confnet.save(
            str(fs.MODEL_DIR / f'{dataset_name}_{confnet.identifier}_{start_time.format(fs.DATE_FORMAT)}'))

Epoch 1/50
45/45 [==============================] - 1s 18ms/step - loss: 2.9641 - val_loss: 1.5216
Epoch 2/50
45/45 [==============================] - 0s 8ms/step - loss: 1.2207 - val_loss: 0.6916
Epoch 3/50
45/45 [==============================] - 0s 8ms/step - loss: 0.6627 - val_loss: 0.4

# Creating the Baseline Models

In [ ]:
from deepalign.alignments.processmining import OptimalCostAligner
from deepalign.alignments.processmining import HeuristicsMinerAligner
from deepalign.alignments.processmining import InductiveMinerAligner

In [ ]:
datasets = sorted([f.name for f in fs.get_event_log_files()])

In [ ]:
aligners = [OptimalCostAligner, HeuristicsMinerAligner, InductiveMinerAligner]

In [ ]:
for aligner_class in tqdm(aligners):
    for dataset_name in tqdm(datasets):
        dataset = Dataset(dataset_name)
        aligner = aligner_class()
        aligner.fit(dataset)
        file_name = f'{dataset_name}_{aligner.abbreviation}'
        aligner.save(file_name)